# Imports

In [73]:
import stlearn as st
import numpy as np
from sklearn.decomposition import NMF
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [60]:
import data_loaders, loss_functions, preprocessing

# Load Data

In [3]:
dl = data_loaders.StlearnLoader()

In [32]:
data = dl.load_from_scanpy_object()
print("X is an annotated data matrix of shape n_obs × n_vars.\nRows correspond to cells and columns to genes.")
data

/usr/local/lib/python3.8/site-packages/anndata/_core/anndata.py:1830: UserWarning: Variable names are not unique. To make them unique, call `.var_names_make_unique`.
  utils.warn_names_duplicates("var")


X is an annotated data matrix of shape n_obs × n_vars.
Rows correspond to cells and columns to genes.


AnnData object with n_obs × n_vars = 4035 × 36601
    obs: 'in_tissue', 'array_row', 'array_col', 'imagecol', 'imagerow'
    var: 'gene_ids', 'feature_types', 'genome'
    uns: 'spatial'
    obsm: 'spatial'

# Preprocessing

In [33]:
data = preprocessing.filtering(data=data, min_counts=100, min_cells=1000)

'New shape after filtering: (4035, 8952)'

# Split to Train and Test 

# EDA 

In [58]:
def count_zeros(data_):
    if not isinstance(data_.X, np.ndarray):
        np_data = data_.X.toarray()
    else:
        np_data = data_.X
    print(f'Matrix shape: {np_data.shape}, # of elements: {np_data.shape[0]*np_data.shape[1]}')
    print(f'# zeros in the matrix: {np.count_nonzero(np_data==0)} ({100*np.count_nonzero(np_data==0)/(np_data.shape[0]*np_data.shape[1])}%)')
    print(f'Median # zeros in each row: {np.median(np.count_nonzero(np_data==0, axis=1))}')
    print(f'Median # zeros in each col: {np.median(np.count_nonzero(np_data==0, axis=0))}')

# Imputation 

In [65]:
R = data.X
R.shape

(4035, 8952)

## SKlearn NMF 

In [61]:
model = NMF(n_components=2, init='random', random_state=0)
model.fit(R)

[[0.27109882 0.587985   0.26203266 ... 4.9851246  1.3424892  3.8723156 ]
 [0.22746569 0.56064945 0.2866197  ... 0.574542   0.4413565  0.55843014]]


/usr/local/lib/python3.8/site-packages/sklearn/decomposition/_nmf.py:1637: ConvergenceWarning: Maximum number of iterations 200 reached. Increase it to improve convergence.
  warnings.warn(


In [79]:
U = model.transform(R)
V = model.components_
print(f'U shape: {U.shape}\nV shape: {V.shape}')

U shape: (4035, 2)
V shape: (2, 8952)


In [68]:
R_trans = model.inverse_transform(U)
R_trans.shape

(4035, 8952)

In [78]:
rmse = loss_functions.RMSE(y_true=R.toarray(), y_pred=R_trans)
print(f'RMSE = {rmse}')

RMSE = 0.4527934193611145


## Neural Matrix Factorization

## STLearn SME Imputation 

In [36]:
data_SME = data.copy()
count_zeros(data_=data_SME)

Matrix shape: (4035, 8952), # of elements: 36121320
# zeros in the matrix: 15244763 (42.20433527899866%)
Median # zeros in each row: 3652.0
Median # zeros in each col: 1824.5


In [37]:
from pathlib import Path
TILE_PATH = Path("/tmp/tiles")
TILE_PATH.mkdir(parents=True, exist_ok=True)

st.pp.tiling(data_SME, TILE_PATH)

# this step uses deep learning model to extract high-level features from tile images
# may need few minutes to be completed
st.pp.extract_feature(data_SME)

Tiling image: 100%|██████████ [ time left: 00:00 ]
2022-04-09 15:49:30.764364: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-04-09 15:49:30.764627: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-04-09 15:49:37.722896: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-04-09 15:49:37.722963: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-04-09 15:49:37.722994: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (275f8f6d768c): /proc/driver/nvidia/version does not exist
2022-04-09 15

Extract feature:   3%|▎          [ time left: 06:01 ]/usr/local/lib/python3.8/site-packages/stlearn/image_preprocessing/feature_extractor.py:68: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  feature_df[spot] = features
/usr/local/lib/python3.8/site-packages/stlearn/image_preprocessing/feature_extractor.py:68: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  feature_df[spot] = features
Extract feature:   3%|▎          [ time left: 05:56 ]/usr/local/lib/python3.8/site-packages/stlearn/image_preprocessing/feature_extractor.py:68: Perf

Extract feature:   3%|▎          [ time left: 05:39 ]/usr/local/lib/python3.8/site-packages/stlearn/image_preprocessing/feature_extractor.py:68: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  feature_df[spot] = features
/usr/local/lib/python3.8/site-packages/stlearn/image_preprocessing/feature_extractor.py:68: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  feature_df[spot] = features
Extract feature:   3%|▎          [ time left: 05:35 ]/usr/local/lib/python3.8/site-packages/stlearn/image_preprocessing/feature_extractor.py:68: Perf

Extract feature:   4%|▍          [ time left: 05:33 ]/usr/local/lib/python3.8/site-packages/stlearn/image_preprocessing/feature_extractor.py:68: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  feature_df[spot] = features
/usr/local/lib/python3.8/site-packages/stlearn/image_preprocessing/feature_extractor.py:68: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  feature_df[spot] = features
Extract feature:   4%|▍          [ time left: 05:35 ]/usr/local/lib/python3.8/site-packages/stlearn/image_preprocessing/feature_extractor.py:68: Perf

Extract feature:   4%|▍          [ time left: 05:35 ]/usr/local/lib/python3.8/site-packages/stlearn/image_preprocessing/feature_extractor.py:68: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  feature_df[spot] = features
/usr/local/lib/python3.8/site-packages/stlearn/image_preprocessing/feature_extractor.py:68: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  feature_df[spot] = features
Extract feature:   4%|▍          [ time left: 05:32 ]/usr/local/lib/python3.8/site-packages/stlearn/image_preprocessing/feature_extractor.py:68: Perf

...

/usr/local/lib/python3.8/site-packages/stlearn/image_preprocessing/feature_extractor.py:68: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  feature_df[spot] = features
Extract feature:  99%|█████████▊ [ time left: 00:05 ]/usr/local/lib/python3.8/site-packages/stlearn/image_preprocessing/feature_extractor.py:68: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  feature_df[spot] = features
/usr/local/lib/python3.8/site-packages/stlearn/image_preprocessing/feature_extractor.py:68: PerformanceWarning: DataFrame is highly fragmented.  This

/usr/local/lib/python3.8/site-packages/stlearn/image_preprocessing/feature_extractor.py:68: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  feature_df[spot] = features
Extract feature:  99%|█████████▉ [ time left: 00:03 ]/usr/local/lib/python3.8/site-packages/stlearn/image_preprocessing/feature_extractor.py:68: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  feature_df[spot] = features
/usr/local/lib/python3.8/site-packages/stlearn/image_preprocessing/feature_extractor.py:68: PerformanceWarning: DataFrame is highly fragmented.  This

/usr/local/lib/python3.8/site-packages/stlearn/image_preprocessing/feature_extractor.py:68: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  feature_df[spot] = features
Extract feature: 100%|█████████▉ [ time left: 00:01 ]/usr/local/lib/python3.8/site-packages/stlearn/image_preprocessing/feature_extractor.py:68: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  feature_df[spot] = features
/usr/local/lib/python3.8/site-packages/stlearn/image_preprocessing/feature_extractor.py:68: PerformanceWarning: DataFrame is highly fragmented.  This

Extract feature: 100%|██████████ [ time left: 00:00 ]


The morphology feature is added to adata.obsm['X_morphology']!


In [43]:
# apply stSME to normalise log transformed data
data_SME_copy = st.spatial.SME.SME_impute0(data_SME, copy=True)
# count_zeros(data_=data_SME)

KeyError: 'X_pca'